### Fundamentals of Natural Language Processing
# Negation and Uncertainty Detection using a Machine-Learning Based Approach

*Authors:*

> *Anna Blanco, Agustina Lazzati, Stanislav Bultaskii, Queralt Salvadó*

*Aims:*
> Rewrite for DL

In [2]:
# Import necessary libraries and functions
import json
import spacy
from collections import defaultdict
import re
import pandas as pd

,sentence_id,token_id,word,lemma,pos,prefix,suffix,is_punct,is_redacted,dep,head_pos,in_single_word_cues,in_affixal_cues,ends_with_ment,neg_cue_label,neg_scope_label
0,0,0,,,SPACE,,,0,0,dep,SPACE,0,0,0,0,0
1,1,0,nº,nº,NOUN,nº,nº,0,0,det,NOUN,0,0,0,0,0
2,1,1,historia,historia,NOUN,his,ria,0,0,ROOT,NOUN,0,0,0,0,0
3,1,2,clinica,clinico,ADJ,cli,ica,0,0,amod,NOUN,0,0,0,0,0
4,1,3,:,:,PUNCT,:,:,1,0,punct,NOUN,0,0,0,0,0
